# 지도 시각화 방법
[**kaggle -Tutorial:All about folium, pydeck**](https://www.kaggle.com/mbnb8317/ds4c-tutorial-all-about-folium-pydeck)의 내용을 정리하였습니다.


* 지도 시각화에 필요한 모듈들을 불러옵니다.  
    folium은 터미널에서 **pip install folium** 코드로 설치 가능합니다.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import seaborn as sns
import matplotlib.pyplot as plt

import folium 
from folium import plugins
# from folium.plugins import HeatMap

import json 
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

* 지도 시각화에 사용할 데이터를 불러옵니다.  
현재 코드가 위치한 폴더는 **C:\Users\tmdgp\python\covid19_visualize** 이고 데이터가 위치한 폴더는 **C:\Users\tmdgp\python\covid19**이기 때문에 path를 다음과 같이 설정하였습니다. 본인에게 알맞은 path로 재설정합니다.

In [2]:
path = os.getcwd()
fpopl = pd.read_csv(path[:-10]+'\\fpopl.csv')
adstrd_master = pd.read_csv(path[:-10]+'\\adstrd_master.csv')
region = pd.read_csv(path[:-10]+'\\COVID_19\\Region.csv')
patient = pd.read_csv(path[:-10]+'\\COVID_19\\PatientInfo.csv')

### 1. 가장 기초적인 지도 띄우기  
**.Map** 코드 안에 input으로 **tiles='Stamen Terrain'** 또는 **tiles='Stamen Toner'** 등을 사용하면 다양한 모습의 지도를 사용할 수 있습니다.  

**.Fullscreen**은 전체화면으로 키울 수 있는 버튼을 만듭니다.  

**.MousePosition**은 오른쪽 아래에 현재 마우스가 위치한 좌표값을 표시합니다.

In [4]:
m = folium.Map([36, 128], zoom_start=7) 

plugins.Fullscreen(position='topright',  # Full screen
                   title='Click to Expand', 
                   title_cancel='Click to Exit', 
                   force_separate_button=True).add_to(m)

plugins.MousePosition().add_to(m) ## you can easily get coordinates.

m

### 2. 지역별 확진자 수를 지도에 표시

확진자 정보를 지도에 표시하기 위해 각 시군구의 위도, 경도 정보를 나타내는 region과 확진자 정보를 나타내는 patient를 묶어 하나의 데이터로 표현합니다.  

데이터를 묶는 기준은 province와 city값으로 하여 동일한 시군구에 해당하는 위도와 경도를 찾아 확진자 정보 옆에 추가합니다.

In [3]:
regional_patient = pd.merge(patient[['patient_id','confirmed_date','sex','age','province','city']],region[['province','city','latitude','longitude']], how = 'left', on = ['province','city'])

.dropna()로 결측값(NaN)이 있는 데이터(위도, 경도값이 없는 데이터)를 제거합니다.  

MarkerCluster를 사용하여 각 확진자가 발생한 위치의 위도 경도에 점을 찍습니다.

In [5]:
regional_count = regional_patient[['latitude','longitude']].dropna()
m = folium.Map([36, 128], zoom_start=7) 

plugins.Fullscreen(position='topright',  # Full screen
                   title='Click to Expand', 
                   title_cancel='Click to Exit', 
                   force_separate_button=True).add_to(m)

plugins.MousePosition().add_to(m) ## you can easily get coordinates.
plugins.MarkerCluster(regional_count).add_to(m)

m

### 3. 성별을 하위그룹으로 하여 따로 좌표 찍기

하위그룹으로 나누어 표현하고자 하는 정보를 각각의 데이터로 따로 생성합니다.  

아래에서는 성별이 여성인 경우와 남성인 경우로 나누어 각각의 확진자 좌표값을 데이터로 생성하였습니다.

In [6]:
male_patient = regional_patient\
.query('sex in ("male")')[['latitude','longitude']].dropna()

female_patient = regional_patient\
.query('sex in ("female")')[['latitude','longitude']].dropna()

add_child로 initial이라는 기본 플러그인을 추가합니다.  

FeatureGroupSubGroup으로 여성과 남성에 대한 그룹을 각각 생성하고 새로 추가하고자 하는 카테고리를 추가합니다.  

추가된 카테고리 각각에 MarkerCluster을 사용하여 위에서 생성한 데이터를 알맞게 넣어줍니다.  

LayerControl 코드로 카테고리를 선택할 수 있는 창을 생성합니다. collapsed를 True로 설정하면 카테고리 선택창이 숨겨집니다.

In [13]:
m = folium.Map([36, 128], zoom_start=7) 

initial = folium.plugins.MarkerCluster(control=False)

m.add_child(initial)


female_g = plugins.FeatureGroupSubGroup(initial, 'Female')
m.add_child(female_g)

male_g = plugins.FeatureGroupSubGroup(initial, 'Male') 
m.add_child(male_g)


plugins.MarkerCluster(female_patient).add_to(female_g)

plugins.MarkerCluster(male_patient).add_to(male_g)



folium.LayerControl(collapsed=True).add_to(m)

m

### 4. 히트맵 그래프

대구의 경우 확진자가 급격히 증가할 당시의 데이터가 제대로 확보되지 않아 patient에 해당 정보가 없어 미포함되어있습니다.  

heatmap의 다양한 parameter들에 대한 정보는 [해당 링크](https://www.kaggle.com/mbnb8317/ds4c-tutorial-all-about-folium-pydeck)(4.Make a Heat map)참고하세요.   

[.groupby](https://rfriend.tistory.com/383)란 전체 데이터를 정해진 기준별로 그룹으로 나누고, 해당 그룹의 데이터를 모두 하나로 합치는 함수입니다. [reset_index](https://kongdols-room.tistory.com/123)에 대한 설명을 참고하세요.


In [16]:
heat_data = regional_patient\
.groupby(['latitude','longitude'])['patient_id'].count().reset_index()\
.values\
.tolist()

m = folium.Map([36, 128], zoom_start=7)#, tiles = 'stamentoner')

folium.plugins.HeatMap(heat_data).add_to(m)

m

### 5. 시간에 따른 히트맵

시간별 히트맵 그래프 생성에 필요한 데이터를 생성합니다. 위의 단순 히트맵 그래프에서 위도와 경도만을 데이터로 사용하였다면, 시간에 따른 히트맵 그래프를 생성할 때에는 확진날짜 데이터를 추가로 사용합니다. 시간이 지남에 따른 확진자수를 표현하기 위해 sort를 사용하여 정렬합니다.  

그래프를 그릴 날짜의 범위를 설정합니다. pandas에서 기본으로 제공하는 date_range를 사용하고, 확진날짜의 최솟값부터 최댓값을 범위로 설정합니다.  

drop_duplicates()를 사용하여 중복되는 값을 제거합니다. 날짜에 따른 위도 경도 데이터프레임을 생성하는 부분의 np.tile 코드에 대한 설명이 필요하다면 [다음 링크](https://rfriend.tistory.com/288)를 참고하세요. cumsum은 누적데이터로 변환하는 함수입니다.  

정리된 데이터를 사용하여 각 날짜별 누적된 위도,경도 데이터를 사용하여 리스트를 생성합니다.

In [17]:
regional_patient_bytime = regional_patient\
.groupby(['confirmed_date','latitude','longitude'])['patient_id'].count().reset_index()\
.sort_values(by = 'confirmed_date')

date_rng = pd.date_range(regional_patient_bytime['confirmed_date'].min(), regional_patient_bytime['confirmed_date'].max())
date_rng = [str(i)[:10] for i in date_rng]

all_date_region = pd.DataFrame({'confirmed_date' : np.repeat(date_rng,region[['latitude','longitude']].drop_duplicates().shape[0]),
              'latitude' : np.tile(list(region['latitude'].drop_duplicates()), len(date_rng)),
              'longitude' : np.tile(list(region['longitude'].drop_duplicates()), len(date_rng))})

all_date_region = pd.merge(all_date_region, regional_patient_bytime, on = ['confirmed_date','latitude','longitude'], how = 'left').fillna(0)
all_date_region['cumsum'] = all_date_region.groupby(['latitude','longitude'])['patient_id'].cumsum()

all_date_region = all_date_region[all_date_region['cumsum'] != 0]

time_index, data = [], []

for date in all_date_region['confirmed_date'].unique():
    time_index += [date[6:].replace('-','/')]
    temp_list = all_date_region.query('confirmed_date == @date')[['latitude','longitude','cumsum']].values.tolist()
    data += [temp_list]
    del temp_list

위에서 생성한 data와 time_index를 사용하여 지도를 생성합니다.  

주석 처리된 부분은 각 버튼에 대한 설명박스를 추가하는 부분입니다.

In [19]:
m = folium.Map([36, 128], zoom_start=7)#, tiles = 'Mapbox Bright')

hm = folium.plugins.HeatMapWithTime(data, index=time_index, auto_play=False, min_opacity=0.3, radius = 25, )

hm.add_to(m)
"""
for bottom, click in zip([60,90,120,150,180],['Loop','Forward','Play','Reverse','Backward']):
    
    # Thank you for html code. https://www.kaggle.com/poonsfc5/ds4c-covid-19-in-korea-eda-with-geo-data
    name = '''
            <div style="position: fixed; bottom: '''+str(bottom)+'''px; left: 50px; width: 100px; height: 29px; 
                        background-color: white; border:1px solid grey; z-index:9999; font-size:11px;text-align:center;"
                        >&nbsp; <br><b>Click : '''+ click +'''</b></div>'''  

    m.get_root().html.add_child(folium.Element(name))
"""
m

### 6. 시간에 따른 원형 표시법(timestampedGeojson)

timestampedGeojson의 경우 time stamp에 시간값을 필요로 합니다. 설정하지 않는 경우 9시 정각으로 자동 설정됩니다.
feature 설정에 대한 설명은 [해당링크](https://www.kaggle.com/mbnb8317/ds4c-tutorial-all-about-folium-pydeck)를 참고합니다.

In [24]:
df_dropna = regional_patient.dropna()
df_dropna['timestamp'] = [i+'T00:00:00' for i in df_dropna['confirmed_date']]

points = []
for date in sorted(df_dropna['confirmed_date'].unique()):
    temp = df_dropna.query('confirmed_date == @date')
    
    temp_dict = {}
    temp_dict['coordinates'] = temp[['longitude','latitude']].values.tolist()
    temp_dict['dates'] = temp['timestamp'].values.tolist()
    
    points += [temp_dict]

features = [ { 'type': 'Feature', 
                  'geometry': { 'type': 'MultiPoint', 
                               'coordinates': point['coordinates'], }, 
                  'properties': { 'times': point['dates'],
                                 'icon' : 'circle'}
             } for point in points ] 

In [25]:
m = folium.Map( location=[36, 128], zoom_start=7 )

plugins.TimestampedGeoJson({ 
    'type': 'FeatureCollection', 
    'features': features, 
}, period='P1D', auto_play = False).add_to(m)

m

추가로 참고할 수 있는 사이트 https://dailyheumsi.tistory.com/85